In [1]:
import importlib
import test_extract
import graph
importlib.reload(test_extract)
importlib.reload(graph)
import time
import matplotlib.pyplot as plt

In [17]:
start = time.time()
data = test_extract.Data("./data/CIS_enrollment.csv", ["Statistical Science"])
end = time.time()
print(end - start)

0.396024227142334


# Data model test

In [18]:
len(data.courses),len(data.students)

(1087, 218)

In [9]:
print(list(data.students.values())[0])

{'id': '2109188', 'major': 'Information Science', 'courses': ['Multivariable Calculus Engrs', 'Cooperative Workshop CHEM 2090', 'Intro to Cognitive Science', 'Data Struct & Functional Progr', 'Networks II: Market Design', 'Discrete Structures', 'Data-Driven Web Applications', 'Programming Practicum', 'Probability Models & Inference', 'Physics II: Electromagnetism', 'Engineering Seminar', 'Intro Computing Using Python', 'Operating Systems', 'Foundations of Artif Inllgnce', 'Prac in A I', 'Intro to Natural Lang Process', 'Physics I: Mechanics & Heat', 'Teams and Technology', 'Systems Programming', 'Information Retrieval', 'Engineering General Chemistry', 'Cooperative Workshop MATH 1920', 'Calculus For Engineers', 'Obj-Oriented Prog & Data Struc', 'Intro Microeconomics', 'Text Mining Hist & Literature', 'Networks', 'Intro Analysis of Algorithms', 'Engineering Communications', 'Bowling', 'Introduction To Sociology', 'Computing Cultures', 'Linear Algebra for Engineers', 'Archery', 'Earthqua

In [10]:
print(list(data.courses["CS1110"].students)[0])

{'id': '1790711', 'major': 'Information Science', 'courses': ['Introduction to Data Science', 'Obj-Oriented Prog & Data Struc', 'Beg/Int. Volleyball', 'Communication and Technology', 'Calculus II', 'Intro to Cognitive Science', 'Adv Human Computer Interaction', 'Our Solar System', 'Engineering in Reality', 'Intro Design & Prog for Web', 'Discrete Structures', 'Intro Bio:Ecol&the Environment', 'Mountain Biking', 'Beginning Snowboarding', 'Analytics-driven Game Design', 'Racquetball', 'Linear Algebra', 'Data-Driven Web Applications', 'Intro Computing Using Python', 'Information Retrieval', 'Human-Computer Interaction Des', 'Networks', 'Human Percept:Appl Comp Graphi', 'Science Writing for Media', 'American Cinema', 'American Capitalism', 'Intermed Design&Prog for Web', 'Intro Game Design', 'UNIX Tools and Scripting', 'Small Boat Sailing', 'Inventing an Info Society', 'Intro Oceanography with Lab', 'Intro to U.S. Labor History'], 'terms': [('ENGRG3400', (2015, 2)), ('PE1460', (2012, 2)), 

# Graph model test

In [19]:
g = graph.Graph()

In [20]:
start = time.time()
c = 1
for cname1 in data.courses:
    if c % 250 == 0:
        print(c)
    for cname2 in data.courses:
        if cname2 == cname1:
            continue
        course = data.courses[cname1]
        count = 0
        for s in course.students:
            if cname2 in s.term_numbers and s.term_numbers[cname2] == s.term_numbers[cname1]:
                count += 1
        if count > 0:
            g.addEdge(cname1, cname2, count)
    c += 1
end = time.time()
print(end - start)

250
500
750
1000
1.5165860652923584


In [25]:
len(g.edges),len(g.filterEdges(lambda e: e.weight > 30))

(111090, 1152)

In [21]:
g.export_json("data/stsci_c")

done! exported as data/stsci_post.json


In [27]:
g.export_json("data/cs_co_30",30)

done! exported as data/cs_co_30.json


In [32]:
g.export_graph("test4",300)

done! exported as test4.dot


In [36]:
x = [min(e.weight, 100) for e in g.edges]
num_bins = 25
n, bins, patches = plt.hist(x, num_bins, facecolor='blue', alpha=0.5)
plt.xlabel("edge weight")
plt.ylabel("frequency")
plt.title("distribution of edge weights (capped at 100)")
plt.show()

AttributeError: 'tuple' object has no attribute 'weight'

In [121]:
len(g.edges)

159731

In [134]:
edges_dict = {}
for e in g.edges:
    edges_dict[(e.src, e.dest)] = e.weight
print(len(edges_dict))

159731


In [135]:
for k in list(edges_dict.keys()):
    k1, k2 = k
    if k in edges_dict and (k2,k1) in edges_dict:
        w1, w2 = edges_dict[k], edges_dict[(k2, k1)]
        if w1/w2 > 4:
            del edges_dict[(k2, k1)]
        elif w2/w1 > 4:
            del edges_dict[k]
print(len(edges_dict))

156744


In [139]:
for k in list(edges_dict.keys()):
    if edges_dict[k] < 5:
        del edges_dict[k]
print(len(edges_dict))    

14225


In [141]:
n = set()
for k in list(edges_dict.keys()):
    n.add(k[0])
    n.add(k[1])
print(len(n))

922


In [142]:
len(g.nodes)

3625

In [20]:
len(set([x.src for x in g.filterEdges(lambda e: e.weight > 30)]+[x.dest for x in g.filterEdges(lambda e: e.weight > 30)]))

174